Sütun başlıkları hakkında.
Country: Ülke
Year: Yıl
Status: Gelişmiş ve gelişmekte olan
Life expectancy: Yaşta yaşam beklentisi
Adult Mortality: Her iki cinsiyette yetişkin ölüm oranları
infant deaths: Bebek ölümlerinin sayısı
Alcohol: Kişi başına göre alkol tüketimi
percentage expenditure: Kişi başı GSYİH
Hepatitis B: 1 yaşındakiler arasında hepatit B aşılama kapsamı
Measles: Kızamık - 1000 kişi başına bildirilen vaka sayısı
BMI: Ortalama vücut kitle indeksi
under-five deaths: Beş yaş altı ölümlerin sayısı
Polio: 1 yaşındakiler arasında çocuk felci aşılama kapsamı
Total expenditure: Sağlık için toplam devlet harcalamaları
Diphtheria: 1 yaşındakiler arasında difteri, tetanoz, toksoidi ve boğmaca aşılama kapsamı
HIV/AIDS: 0-4 yaş 1000 canlı doğumda ölüm
GDP: Kişi başına Gayri Safi Yurtiçi Hasıla (ABD Doları cinsinden)
Population: Ülkenin nüfusu
thinness 1-19 years: 10-19 Yaş arası çocuklar ve ergenler arasında zayıflık prevalansı
thinness 5-9 years: 5-9 Yaş arası çocuklarda zayıflık prevalansı (%)
Income composition of resources: Kaynakların gelir bileşimi açısından İnsani Gelişme Endeksi
Schooling: Eğitim yılı sayısı (yıl)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [ ]:
dizi = pd.read_csv('../input/life-expectancy-data/Life_Expectancy_Data.csv', header = 0)
dizi.info()

In [ ]:
dizi.head()

In [ ]:
dizi.describe()

In [ ]:
#Gözlem ve değişken sayılarını inceliyoruz.
dizi.shape

In [ ]:
#Her bir değişkenin veri tiplerini inceliyoruz.
dizi.dtypes

In [ ]:
#Şimdi her kolonda kaç tane eksik değer var onları inceliyoruz.
dizi.isnull().sum()

In [ ]:
#Eksik değerlerin yerini görselleştiriyoruz.
fig, ax = plt.subplots(figsize = (9, 5))
sns.heatmap(dizi.isnull(), cmap = "cubehelix_r", yticklabels='')
plt.show()

In [ ]:
#Her özellik için boş olan değerleri siliyoruz.
dizi.dropna(inplace=True)
dizi.isnull().sum()

In [ ]:
#Meta verileri (ülke ve yıl) atıyoruz.
dizi = dizi.drop('Country', axis=1)
dizi = dizi.drop('Year', axis=1)

In [ ]:
#Sütun isimlerini standart bir hale getiriyoruz.
orjinal_sutunlar = list(dizi.columns) 
yeni_sutunlar = [] 
for sutun in orjinal_sutunlar:     
    yeni_sutunlar.append(sutun.strip().replace('  ', ' ').replace(' ', '_').lower()) 

dizi.columns = yeni_sutunlar

In [ ]:
# Box Plot kullanarak her bir özelliğin verilerini görselleştiriyoruz.
sutun_isimleri = {'life_expectancy':1,'adult_mortality':2,'infant_deaths':3,'alcohol':4,'percentage_expenditure':5,'hepatitis_b':6,'measles':7,'bmi':8,
            'under-five_deaths':9,'polio':10,'total_expenditure':11,'diphtheria':12,'hiv/aids':13,'gdp':14,'population':15,'thinness_1-19_years':16,
            'thinness_5-9_years':17,'income_composition_of_resources':18,'schooling':19}

plt.figure(figsize=(18,30))

for variable,i in sutun_isimleri.items():
                     plt.subplot(5,4,i)
                     plt.boxplot(dizi[variable],whis=1.5)
                     plt.title(variable)

plt.show()
dizi.shape

In [ ]:
#Çeyrekler arası aralığı (IQR) kullanarak aykırı değerleri kaldırıyoruz.
Q1 = dizi.quantile(0.25)
Q3 = dizi.quantile(0.75)
IQR = Q3 - Q1

dizi = dizi[~((dizi < (Q1 - 1.5 * IQR)) |(dizi > (Q3 + 1.5 * IQR))).any(axis=1)]

#Durumu boolean değişkenleriyle değiştiriyoruz.
dizi["status"].replace({"Developing": 1, "Developed": 0}, inplace=True)
dizi.shape

In [ ]:
#Korelasyonları görselleştirmek için ısı haritasını çiziyoruz.
plt.figure(figsize = (14, 12))
sns.heatmap(dizi.corr(), annot = True)
plt.title('Özellikler arasındaki ilişki');

In [ ]:
#Özellikleri etiketlerden ayırıyoruz.
X = dizi.iloc[:,1:].values
y = dizi.iloc[:,0].values

In [ ]:
#Verileri normalleştiriyoruz.
X_std= StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)

#Kovaryans matrisini hesaplıyoruz.
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)

In [ ]:
# Özdeğerleri ve özvektörleri hesaplayın.
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]

In [ ]:
#Kümülatif varyansın grafiğini çiziyoruz.
pca = PCA(n_components=19).fit(X_std)
plt.figure(figsize=(12, 4))
plt.plot(np.cumsum(pca.explained_variance_ratio_), label='Kümülatif varyans')
plt.xlim(0,18,1)
plt.xlabel('Bileşen sayısı')
plt.ylabel('Kümülatif açıklanan varyans')
plt.legend(loc='best')
plt.grid(color='#E3E3E3')
plt.xticks(np.arange(0, 19, 1.0));

In [ ]:
#Hedef değişkenler ile ilgili özelliklerin değerlerini alıyoruz.
ozellikler = dizi['income_composition_of_resources'].values.reshape(-1,1)
etiketler = dizi['life_expectancy'].values.reshape(-1,1)

#Veriyi normalleştiriyoruz.
min_max_scaler = MinMaxScaler()
ozellikler = min_max_scaler.fit_transform(ozellikler)

#Veri setini eğitim ve test setine bölüyoruz.
ozellikler_egitim, ozellikler_test, etiketler_egitim, etiketler_test = train_test_split(ozellikler, etiketler, train_size = 0.7, test_size = 0.3)

In [ ]:
linear_model = LinearRegression()
#Modeli eğitiyoruz.
linear_model.fit(ozellikler_egitim, etiketler_egitim);

In [ ]:
#Modeli test ediyoruz.
linear_model_score = linear_model.predict(ozellikler_test)

plt.figure(figsize=(10, 6))
plt.scatter(ozellikler_test, etiketler_test,  color='black')
plt.plot(ozellikler_test, linear_model_score, color='blue', linewidth=2)
plt.xlabel('Kaynakların gelir bileşimi')
plt.ylabel('Yaşam beklentisi')
plt.show()

print('Katsayılar: \n', linear_model.coef_)
print("Ortalama karesel hata: %.2f" % mean_squared_error(etiketler_test,linear_model_score))

In [ ]:
ozellikler = dizi.iloc[:, 1:].values
etiketler = dizi.iloc[:,0]

min_max_scaler = MinMaxScaler()
ozellikler = min_max_scaler.fit_transform(ozellikler)

ozellikler_egitim, ozellikler_test, etiketler_egitim, etiketler_test = train_test_split(ozellikler, etiketler, train_size = 0.7, test_size = 0.3)

In [ ]:
linear_model.fit(ozellikler_egitim, etiketler_egitim);

linear_model_score = linear_model.predict(ozellikler_test)

print('Katsayılar: \n', linear_model.coef_)
print("Ortalama karesel hata: %.2f" % mean_squared_error(etiketler_test, linear_model_score))

In [ ]:
#Yaşam beklentisinin ortalamasını hesaplıyoruz.
dizi_ort = dizi['life_expectancy'].mean()
dizi_lr = dizi.copy()

# Yaşam beklentisi ortalamadan büyük ise 1, değil ise 0
dizi_lr['life_expectancy'] = (dizi_lr['life_expectancy'] > dizi_ort).astype(int)

# Separate the features from the labels.
ozellikler_lr = dizi_lr.iloc[:, 1:].values
etiketler_lr = dizi_lr.iloc[:,0]

min_max_scaler = MinMaxScaler()
ozellikler_lr = min_max_scaler.fit_transform(ozellikler_lr)

# Split the dataset in training and test set.
ozellikler_egitim_lr, ozellikler_test_lr, etiketler_egitim_lr, etiketler_test_lr = train_test_split(ozellikler_lr, etiketler_lr, train_size = 0.7, test_size = 0.3)

In [ ]:
logistic_model = LogisticRegression(solver='liblinear')

#Modeli eğitiyoruz.
logistic_model.fit(ozellikler_egitim_lr, etiketler_egitim_lr);

In [ ]:
logistic_score = logistic_model.predict(ozellikler_test_lr)

confusion_matrix = confusion_matrix(etiketler_test_lr, logistic_score)

#Karışıklık matrisi ısı haritası
class_names=[0,1]
fig,ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
sns.heatmap(pd.DataFrame(confusion_matrix),cmap='YlGnBu',annot=True,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.xlabel('Tahmin edilen etiket')
plt.ylabel('Gerçek etiket')
plt.show()

In [ ]:
print("Eğitim verisi skor: %.2f" % logistic_model.score(ozellikler_egitim_lr, etiketler_egitim_lr))
print("Test verisi skor: %.2f" % logistic_model.score(ozellikler_test_lr, etiketler_test_lr))

In [ ]:
# 3 farklı derinlikte gerçekleştiriyoruz.
decision_tree_model1 = DecisionTreeRegressor(max_depth=1)
decision_tree_model3 = DecisionTreeRegressor(max_depth=3)
decision_tree_model5 = DecisionTreeRegressor(max_depth=5)

#Eğitim modeli
decision_tree_model1 = decision_tree_model1.fit(ozellikler_egitim, etiketler_egitim)
decision_tree_model3 = decision_tree_model3.fit(ozellikler_egitim, etiketler_egitim)
decision_tree_model5 = decision_tree_model5.fit(ozellikler_egitim, etiketler_egitim)

print("Derinliği 1 olan eğitim verisindeki puan: %.2f" % decision_tree_model1.score(ozellikler_egitim, etiketler_egitim))
print("Derinliği 1 olan test verisindeki puan: %.2f" % decision_tree_model1.score(ozellikler_test, etiketler_test))
print("Derinliği 3 olan eğitim verisindeki puan: %.2f" % decision_tree_model3.score(ozellikler_egitim, etiketler_egitim))
print("Derinliği 3 olan test verisindeki puan: %.2f" % decision_tree_model3.score(ozellikler_test, etiketler_test))
print("Derinliği 5 olan eğitim verisindeki puan: %.2f" % decision_tree_model5.score(ozellikler_egitim, etiketler_egitim))
print("Derinliği 5 olan test verisindeki puan: %.2f" % decision_tree_model5.score(ozellikler_test, etiketler_test))

In [ ]:
random_forest_model = RandomForestRegressor(n_estimators=100,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)

random_forest_model.fit(ozellikler_egitim, etiketler_egitim);

In [ ]:
df = pd.DataFrame(dizi.iloc[:, 1:].columns, columns = ['feature'])
df['fscore'] = random_forest_model.feature_importances_[:, ]

#İlk 10 özelliği göreceli inceliyoruz.
df['fscore'] = df['fscore'] / df['fscore'].max()
df.sort_values('fscore', ascending = False, inplace = True)
df = df[0:19]
df.sort_values('fscore', ascending = True, inplace = True)
ax = df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(15, 10))

# Plot the result.
plt.title('Random Forest özelliği')
plt.xlabel('')
plt.ylabel('')
plt.xticks([], [])
plt.yticks()

#Verileri toplamak için liste oluşturuyoruz.
toplamlar = []

for i in ax.patches:
    toplamlar.append(i.get_width())

# Set individual bar lables using above list.
toplam = sum(toplamlar)

for i in ax.patches:
    # get_width pulls left or right; get_y pushes up or down.
    ax.text(i.get_width(), i.get_y()+.13, \
            str(round((i.get_width()/toplam)*100, 2))+'%', fontsize=10,
color='#505050')
    
plt.show()

In [ ]:
random_forest_score = random_forest_model.predict(ozellikler_test)

print("Eğitim verisindeki puan: %.2f" % random_forest_model.score(ozellikler_egitim, etiketler_egitim))
print("Test verisindeki puan: %.2f" % random_forest_model.score(ozellikler_test, etiketler_test))